# 1. Install and Import Libraries

In [ ]:
!pip install optuna

In [ ]:
import os
import json
import random
from copy import deepcopy
from pathlib import Path
import time
from google.colab import drive
import gc

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    BertForSequenceClassification,
    AlbertForSequenceClassification,
    BertTokenizer,
    get_linear_schedule_with_warmup,
)
from torch.optim import AdamW

import optuna

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

import optuna.visualization.matplotlib as opviz_mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import warnings

from optuna.importance import FanovaImportanceEvaluator
from optuna.importance import get_param_importances

# 2. Environment Settings and Initialization

## 2.1. Set Seed for Reproducibility

In [ ]:
SEED = 42

In [ ]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cuda.matmul.allow_tf32 = False
    torch.backends.cudnn.allow_tf32 = False

In [ ]:
seed_everything(42)

In [ ]:
def seed_worker(worker_id):
    worker_seed = SEED + worker_id
    np.random.seed(worker_seed)
    random.seed(worker_seed)


In [ ]:
def make_generator(seed):
    g = torch.Generator()
    g.manual_seed(seed)
    return g

## 2.2. Set GPU

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

## 2.3. Manage Google Drive

### 2.3.1. Mount Google Drive

In [ ]:
drive.mount('/content/drive')

### 2.3.2. Set Folder Path

In [ ]:
# Datasets
READ_PATH = '/content/drive/MyDrive/Bach_Thesis/Dataset/'

# Models
SAVE_ROOT = "/content/drive/MyDrive/Bach_Thesis/Models/SA_Optuna"

## 2.4. Initialize Global Parameter and Variables

In [ ]:
NUM_LABELS = 2
LABELS = [0,1]
TARGET_NAMES = ['negative' ,'positive']

## 2.5. Initialize Datasets

In [ ]:
def extract_xy(df, text_col="cleaned_content", label_col="sentiment"):
    X = df[text_col].tolist()
    y = df[label_col].tolist()
    return X, y

In [ ]:
df_train_ori = pd.read_csv(f"{READ_PATH}Train2lab.csv")
df_train_ros = pd.read_csv(f"{READ_PATH}Train_ROS2lab.csv")
df_train_ros_ncl = pd.read_csv(f"{READ_PATH}Train_ROS_NCL2lab.csv")
df_val = pd.read_csv(f"{READ_PATH}Validation2lab.csv")
df_test = pd.read_csv(f"{READ_PATH}Test2lab.csv")

In [ ]:
X_train, y_train = extract_xy(df_train_ori)
X_train_ros, y_train_ros = extract_xy(df_train_ros)
X_train_ros_ncl, y_train_ros_ncl = extract_xy(df_train_ros_ncl)
X_val, y_val     = extract_xy(df_val)
X_test, y_test   = extract_xy(df_test)

In [ ]:
class_weights = torch.tensor(
    compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train),
    dtype=torch.float
).to(DEVICE)

class_weights_ros = torch.tensor(
    compute_class_weight(class_weight='balanced', classes=np.unique(y_train_ros), y=y_train_ros),
    dtype=torch.float
).to(DEVICE)

class_weights_ros_ncl = torch.tensor(
    compute_class_weight(class_weight='balanced', classes=np.unique(y_train_ros_ncl), y=y_train_ros_ncl),
    dtype=torch.float
).to(DEVICE)

# 3. Helper Functions

In [ ]:
MAX_LEN = int(df_train_ori['cleaned_content'].str.split().str.len().max())
MAX_LEN

## 3.1. Dataset Class

In [ ]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = list(texts)
        self.labels = list(labels)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
def collate_fn(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    labels = torch.stack([item['labels'] for item in batch])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

## 3.2. Initialization Model and Tokenizer

In [ ]:
def init_model_and_tokenizer(MODEL_NAME, num_labels=NUM_LABELS, hidden_dropout_prob=None, attention_dropout_prob=None):
    low = MODEL_NAME.lower()
    model_kwargs = {}

    if hidden_dropout_prob is not None:
        model_kwargs['hidden_dropout_prob'] = float(hidden_dropout_prob)
    if attention_dropout_prob is not None:
        model_kwargs['attention_probs_dropout_prob'] = float(attention_dropout_prob)

    if "lite" in low:
        model = AlbertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels, **model_kwargs)
        tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
    elif "nusabert" in low:
        model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels, **model_kwargs)
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
    else:
        model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels, **model_kwargs)
        tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

    return model, tokenizer

## 3.4. Create Classification Report

In [ ]:
def create_classification_report(y_true, y_pred):
    cr = classification_report(
        y_true, y_pred,
        labels=LABELS,
        target_names=TARGET_NAMES,
        zero_division=0,
        output_dict=True
    )
    df_cr = pd.DataFrame(cr).transpose().reset_index()
    df_cr = df_cr.rename(columns={'index': 'label'})

    for col in df_cr.select_dtypes(include=['float']).columns:
        df_cr[col] = df_cr[col].apply(lambda x: round(x, 4))

    return df_cr

## 3.5. Create Confusion Matrix

In [ ]:
def create_confusion_matrix(y_true, y_pred, save_path_png):
    cm = confusion_matrix(y_true, y_pred, labels=LABELS)
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=TARGET_NAMES, yticklabels=TARGET_NAMES)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.savefig(save_path_png, dpi=150)
    plt.close()

## 3.6. Training and Validation

In [ ]:
def train_validate_model(model, train_loader, val_loader, optimizer, scheduler, loss_fn, device):
    # TRAIN
    model.train()
    total_train_loss = 0.0

    for batch in tqdm(train_loader, desc="Training", leave=False):
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        loss = loss_fn(outputs.logits, labels)
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_loader)

    # VAL
    model.eval()
    total_val_loss = 0.0
    val_preds, val_labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            loss = loss_fn(outputs.logits, labels)
            total_val_loss += loss.item()

            preds = torch.argmax(outputs.logits, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(val_loader)
    val_f1 = f1_score(val_labels, val_preds, average="weighted")

    return val_f1, avg_train_loss, avg_val_loss

## 3.7. Evaluation

In [ ]:
def evaluate_model(model, loader, device):
    preds, trues = [], []
    model.eval()
    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
            trues.extend(labels.cpu().numpy())
    return preds, trues

In [ ]:
def print_summary(summary):
    print("\n" + "="*40)
    print("✅ Model Training Summary")
    print("="*40)
    print(f"Model Name: {summary['model_name']}")
    print(f"Train Size: {summary['n_train']}")
    print(f"Validation Size: {summary['n_val']}")
    print(f"Test Size: {summary['n_test']}")
    print("-"*40)
    print(f"Train Weighted F1: {summary['train_weighted_f1']:.4f}")
    print(f"Validation Weighted F1: {summary['val_weighted_f1']:.4f}")
    print(f"Test Weighted F1: {summary['test_weighted_f1']:.4f}")
    print("-"*40)
    print(f"Train Accuracy: {summary['train_accuracy']:.4f}")
    print(f"Validation Accuracy: {summary['val_accuracy']:.4f}")
    print(f"Test Accuracy: {summary['test_accuracy']:.4f}")
    print("="*40)

## 3.8. Optuna

In [ ]:
def objective(trial, model_name, X_train, y_train, X_val, y_val, class_weights):
    seed_everything(SEED)

    # SEARCH SPACE
    lr = trial.suggest_float("lr", 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    epochs = trial.suggest_int("epochs", 3, 10)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 0.1, log=True)
    warmup_ratio = trial.suggest_float("warmup_ratio", 0.0, 0.2)
    hidden_dropout_prob = trial.suggest_float("hidden_dropout_prob", 0.1, 0.4, log=True)
    attention_dropout_prob = trial.suggest_float("attention_dropout_prob", 0.1, 0.4, log=True)

    # INITIALIZATIONS
    model, tokenizer = init_model_and_tokenizer(
        model_name,
        num_labels=NUM_LABELS,
        hidden_dropout_prob=hidden_dropout_prob,
        attention_dropout_prob=attention_dropout_prob
    )
    model.to(DEVICE)

    train_loader = DataLoader(
        SentimentDataset(X_train, y_train, tokenizer, max_len=MAX_LEN),
        batch_size=batch_size, shuffle=True, collate_fn=collate_fn,
        worker_init_fn=seed_worker,
        generator=make_generator(SEED)
    )

    val_loader = DataLoader(
        SentimentDataset(X_val, y_val, tokenizer, max_len=MAX_LEN),
        batch_size=batch_size, shuffle=False, collate_fn=collate_fn,
        worker_init_fn=seed_worker,
        generator=make_generator(SEED)
    )

    optimizer = AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    total_steps = len(train_loader) * epochs
    warmup_steps = int(total_steps * warmup_ratio)

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )

    loss_fn = nn.CrossEntropyLoss(weight=class_weights)

    history = []

    # TRAIN OPTUNA
    for epoch in range(epochs):
        val_f1, train_loss, val_loss = train_validate_model(model, train_loader, val_loader, optimizer, scheduler, loss_fn, DEVICE)

        history.append({
            "epoch": epoch + 1,
            "train_loss": float(train_loss),
            "val_loss": float(val_loss),
            "val_weighted_f1": float(val_f1)
        })

        trial.report(val_f1, epoch)

        if trial.should_prune():
            trial.set_user_attr("history", history)
            raise optuna.TrialPruned()

    trial.set_user_attr("history", history)

    del model, tokenizer
    torch.cuda.empty_cache()
    gc.collect()

    return val_f1

In [ ]:
def run_optuna_for_model(model_name, X_train, y_train, X_val, y_val class_weights, n_trials, save_root, dataset_name
):
    model_dir = model_name.replace("/", "_")

    # STORAGE INITIALIZATION
    experiment_dir = Path(save_root) / model_dir / dataset_name
    optuna_dir = experiment_dir / "optuna_study"
    optuna_dir.mkdir(parents=True, exist_ok=True)

    storage_path = optuna_dir / "study.db"
    study_name_path = optuna_dir / "study_name.txt"

    storage_uri = f"sqlite:///{storage_path}"

    # LOAD/CREATE STUDY
    if storage_path.exists() and study_name_path.exists():
        # Resume existing study
        study_name = study_name_path.read_text().strip()
        print(f"Resuming Optuna study: {study_name}")
        study = optuna.load_study(
            study_name=study_name,
            storage=storage_uri
        )
    else:
        study_name = f"opt_{model_dir}_{dataset_name}"
        print(f"Creating Optuna study: {study_name}")

        study_name_path.write_text(study_name)

        sampler = optuna.samplers.TPESampler(seed=SEED)

        study = optuna.create_study(
            study_name=study_name,
            direction="maximize",
            sampler=sampler,
            storage=storage_uri,
            load_if_exists=True
        )

    # OPTUNA OPTIMIZATION (OBJ. FUNCTION)
    study.optimize(
        lambda trial: objective(
            trial,
            model_name,
            X_train, y_train,
            X_val, y_val,
            class_weights
        ),
        n_trials=n_trials
    )

    print(f"BEST weighted F1 = {study.best_value}")
    print("BEST hyperparameters:")
    print(study.best_params)

    return study.best_params

In [ ]:
plt.style.use('default')
warnings.filterwarnings("ignore", category=optuna.exceptions.ExperimentalWarning)


def hp_importance_graph(save_root, model_name, dataset_name):
    model_dir = model_name.replace("/", "_")
    experiment_dir = Path(save_root) / model_dir / dataset_name
    optuna_dir = experiment_dir / "optuna_study"
    storage_path = optuna_dir / "study.db"
    study_name_path = optuna_dir / "study_name.txt"
    storage_uri = f"sqlite:///{storage_path}"

    if storage_path.exists() and study_name_path.exists():
        study_name = study_name_path.read_text().strip()
        print(f"Loading existing Optuna study: {study_name}")

        try:
            loaded_study = optuna.load_study(
                study_name=study_name,
                storage=storage_uri
            )

            # CALCULATE IMPORTANCE
            importance_evaluator = FanovaImportanceEvaluator(seed=SEED)

            ax = opviz_mpl.plot_param_importances(
                loaded_study,
                evaluator=importance_evaluator, evaluator
                target=lambda t: t.values[0],
                target_name='Objective'
            )

            fig = ax.figure

            # PLOT
            num_params = len(ax.patches)
            cmap = plt.colormaps['berlin']
            colors = [cmap(i/num_params) for i in range(num_params)]
            for i, patch in enumerate(ax.patches):
                patch.set_facecolor(colors[i])

            ax.set_facecolor('white')
            fig.set_facecolor('white')

            ax.set_title('')
            fig.texts = []

            fig.suptitle(f'Hyperparameter Importances for {model_name}', fontsize=12, color='black')

            ax.xaxis.label.set_color('black')
            ax.yaxis.label.set_color('black')
            ax.tick_params(axis='x', colors='black')
            ax.tick_params(axis='y', colors='black')

            for text in ax.texts:
                text.set_color('black')

            if ax.legend_:
                ax.legend().remove()

            plt.tight_layout()
            plt.show()

        except Exception as e:
            print(f"An error occurred while plotting the graph: {e}")
    else:
        print(f"Error: Study files not found at expected path: {optuna_dir}")

## 3.9 Final Model Creation

In [ ]:
def pipeline(model_name, X_train, y_train, X_val, y_val, X_test, y_test, best_hp, class_weights, dataset_name, save_root=SAVE_ROOT, device=DEVICE, num_labels=NUM_LABELS
):
    model_dir_name = model_name.replace("/", "_")

    experiment_dir = Path(save_root) / model_dir_name / dataset_name
    model_folder = experiment_dir / "final_model"
    model_folder.mkdir(parents=True, exist_ok=True)
    print("Outputs will be saved to:", model_folder)

    model, tokenizer = init_model_and_tokenizer(
        model_name,
        num_labels=num_labels,
        hidden_dropout_prob=best_hp.get("hidden_dropout_prob"),
        attention_dropout_prob=best_hp.get("attention_dropout_prob")
    )
    model.to(device)

    train_loader = DataLoader(
        SentimentDataset(X_train, y_train, tokenizer, max_len=MAX_LEN),
        batch_size=best_hp.get("batch_size", 32), shuffle=True, collate_fn=collate_fn,
        worker_init_fn=seed_worker,
        generator=make_generator(SEED)
    )
    val_loader = DataLoader(
        SentimentDataset(X_val, y_val, tokenizer, max_len=MAX_LEN),
        batch_size=best_hp.get("batch_size", 32), shuffle=False, collate_fn=collate_fn,
        worker_init_fn=seed_worker,
        generator=make_generator(SEED)
    )
    test_loader = DataLoader(
        SentimentDataset(X_test, y_test, tokenizer, max_len=MAX_LEN),
        batch_size=best_hp.get("batch_size", 32), shuffle=False, collate_fn=collate_fn,
        worker_init_fn=seed_worker,
        generator=make_generator(SEED)
    )

    loss_fn = nn.CrossEntropyLoss(weight=class_weights)

    optimizer = AdamW(
        model.parameters(),
        lr=best_hp.get("lr", 2e-5),
        weight_decay=best_hp.get("weight_decay", 0.0)
    )
    total_steps = len(train_loader) * best_hp.get("epochs", 3)
    scheduler = get_linear_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=int(total_steps * best_hp.get("warmup_ratio", 0.0)),
        num_training_steps=total_steps
    )

    # FINAL MODEL TRAINING
    training_history = []

    for epoch in range(best_hp.get("epochs", 3)):
        val_f1, train_loss, val_loss = train_validate_model(
            model, train_loader, val_loader, optimizer, scheduler, loss_fn, device
        )

        training_history.append({
            "epoch": epoch + 1,
            "train_loss": float(train_loss),
            "val_loss": float(val_loss),
            "val_weighted_f1": float(val_f1)
        })

    model.save_pretrained(model_folder)
    tokenizer.save_pretrained(model_folder)
    print("✅ Saved final model to:", model_folder)

    history_csv_path = model_folder / "training_history.csv"
    pd.DataFrame(training_history).to_csv(history_csv_path, index=False)
    print("✅ Saved training history to:", history_csv_path)

    # EVALUATIONS
    train_preds, train_trues = evaluate_model(model, train_loader, device)
    val_preds, val_trues = evaluate_model(model, val_loader, device)
    test_preds, test_trues = evaluate_model(model, test_loader, device)

    train_cr_df = create_classification_report(train_trues, train_preds)
    val_cr_df = create_classification_report(val_trues, val_preds)
    test_cr_df = create_classification_report(test_trues, test_preds)

    train_cr_df["dataset"] = "train"
    val_cr_df["dataset"] = "validation"
    test_cr_df["dataset"] = "test"

    all_reports_df = pd.concat([train_cr_df, val_cr_df, test_cr_df])
    all_reports_df.to_csv(model_folder / "all_classification_reports.csv", index=False)
    print("✅ Saved classification reports.")
    create_confusion_matrix(test_trues, test_preds, model_folder / "confusion_matrix.png")
    print("✅ Saved confusion matrix.")

    summary = {
        "model_name": model_name,
        "dataset_name": dataset_name,
        "n_train": len(X_train),
        "n_val": len(X_val),
        "n_test": len(X_test),
        "train_weighted_f1": float(f1_score(train_trues, train_preds, average='weighted')),
        "val_weighted_f1": float(f1_score(val_trues, val_preds, average='weighted')),
        "test_weighted_f1": float(f1_score(test_trues, test_preds, average='weighted')),
        "train_accuracy": float((np.array(train_preds) == np.array(train_trues)).mean()),
        "val_accuracy": float((np.array(val_preds) == np.array(val_trues)).mean()),
        "test_accuracy": float((np.array(test_preds) == np.array(test_trues)).mean())
    }

    summary_json_path = model_folder / "summary.json"
    with open(summary_json_path, "w") as f:
        json.dump(summary, f, indent=2)
    pd.json_normalize(summary).to_csv(model_folder / "summary_metrics.csv", index=False)

    print(f"✅ Pipeline Completed for {model_name} [{dataset_name}].")

    del model, tokenizer
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

    return {
        "summary": summary,
        "train_classification_report": train_cr_df,
        "validation_classification_report": val_cr_df,
        "test_classification_report": test_cr_df,
        "model_dir": str(model_folder)
    }

# 4. Run Optuna and Final Model Pipeline

## 4.1. indobenchmark/indobert-base-p1

In [ ]:
mod_indo_base_p1 = 'indobenchmark/indobert-base-p1'

### 4.1.1. Original

In [ ]:
hp_indo_base_p1 = run_optuna_for_model(
    model_name=mod_indo_base_p1,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    n_trials=20,
    class_weights=class_weights,
    save_root=SAVE_ROOT,
    dataset_name="original"
)

In [ ]:
pipeline(
    model_name=mod_indo_base_p1,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    best_hp=hp_indo_base_p1,
    class_weights=class_weights,
    dataset_name="original",
    save_root=SAVE_ROOT,
    device=DEVICE,
    num_labels=NUM_LABELS
)

In [ ]:
hp_importance_graph(
    save_root=SAVE_ROOT,
    model_name=mod_indo_base_p1,
    dataset_name="original"
)

### 4.1.2. ROS

In [ ]:
hp_indo_base_p1_ros = run_optuna_for_model(
    model_name=mod_indo_base_p1,
    X_train=X_train_ros,
    y_train=y_train_ros,
    X_val=X_val,
    y_val=y_val,
    n_trials=20,
    class_weights=class_weights_ros,
    save_root=SAVE_ROOT,
    dataset_name="ROS"
)

In [ ]:
pipeline(
    model_name=mod_indo_base_p1,
    X_train=X_train_ros,
    y_train=y_train_ros,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    best_hp=hp_indo_base_p1_ros,
    class_weights=class_weights_ros,
    dataset_name="ROS",
    save_root=SAVE_ROOT,
    device=DEVICE,
    num_labels=NUM_LABELS
)

In [ ]:
hp_importance_graph(
    save_root=SAVE_ROOT,
    model_name=mod_indo_base_p1,
    dataset_name="ROS"
)

### 4.1.3. ROS+NCL

In [ ]:
hp_indo_base_p1_ros_ncl = run_optuna_for_model(
    model_name=mod_indo_base_p1,
    X_train=X_train_ros_ncl,
    y_train=y_train_ros_ncl,
    X_val=X_val,
    y_val=y_val,
    n_trials=20,
    class_weights=class_weights_ros_ncl,
    save_root=SAVE_ROOT,
    dataset_name="ROS-NCL"
)

In [ ]:
res_indo_2_ros_ncl = pipeline(
    model_name=mod_indo_base_p1,
    X_train=X_train_ros_ncl,
    y_train=y_train_ros_ncl,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    best_hp=hp_indo_base_p1_ros_ncl,
    class_weights=class_weights_ros_ncl,
    dataset_name="ROS-NCL",
    save_root=SAVE_ROOT,
    device=DEVICE,
    num_labels=NUM_LABELS
)

In [ ]:
hp_importance_graph(
    save_root=SAVE_ROOT,
    model_name=mod_indo_base_p1,
    dataset_name="ROS-NCL"
)

## 4.2. indobenchmark/indobert-base-p2

In [ ]:
mod_indo_base_p2 = 'indobenchmark/indobert-base-p2'

### 4.2.1. Original

In [ ]:
hp_indo_base_p2 = run_optuna_for_model(
    model_name=mod_indo_base_p2,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    n_trials=20,
    class_weights=class_weights,
    save_root=SAVE_ROOT,
    dataset_name="original"
)

In [ ]:
pipeline(
    model_name=mod_indo_base_p2,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    best_hp=hp_indo_base_p2,
    class_weights=class_weights,
    dataset_name="original",
    save_root=SAVE_ROOT,
    device=DEVICE,
    num_labels=NUM_LABELS
)

In [ ]:
hp_importance_graph(
    save_root=SAVE_ROOT,
    model_name=mod_indo_base_p2,
    dataset_name="original"
)

### 4.2.2. ROS

In [ ]:
hp_indo_base_p2_ros = run_optuna_for_model(
    model_name=mod_indo_base_p2,
    X_train=X_train_ros,
    y_train=y_train_ros,
    X_val=X_val,
    y_val=y_val,
    n_trials=20,
    class_weights=class_weights_ros,
    save_root=SAVE_ROOT,
    dataset_name="ROS"
)

In [ ]:
pipeline(
    model_name=mod_indo_base_p2,
    X_train=X_train_ros,
    y_train=y_train_ros,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    best_hp=hp_indo_base_p2_ros,
    class_weights=class_weights_ros,
    dataset_name="ROS",
    save_root=SAVE_ROOT,
    device=DEVICE,
    num_labels=NUM_LABELS
)

In [ ]:
hp_importance_graph(
    save_root=SAVE_ROOT,
    model_name=mod_indo_base_p2,
    dataset_name="ROS"
)

### 4.2.3. ROS+NCL

In [ ]:
hp_indo_base_p2_ros_ncl = run_optuna_for_model(
    model_name=mod_indo_base_p2,
    X_train=X_train_ros_ncl,
    y_train=y_train_ros_ncl,
    X_val=X_val,
    y_val=y_val,
    n_trials=20,
    class_weights=class_weights_ros_ncl,
    save_root=SAVE_ROOT,
    dataset_name="ROS-NCL"
)

In [ ]:
res_indo_2_ros_ncl = pipeline(
    model_name=mod_indo_base_p2,
    X_train=X_train_ros_ncl,
    y_train=y_train_ros_ncl,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    best_hp=hp_indo_base_p2_ros_ncl,
    class_weights=class_weights_ros_ncl,
    dataset_name="ROS-NCL",
    save_root=SAVE_ROOT,
    device=DEVICE,
    num_labels=NUM_LABELS
)

In [ ]:
hp_importance_graph(
    save_root=SAVE_ROOT,
    model_name=mod_indo_base_p2,
    dataset_name="ROS-NCL"
)

## 4.3. indobenchmark/indobert-lite-base-p1

In [ ]:
mod_indo_lite_p1 = 'indobenchmark/indobert-lite-base-p1'

### 4.3.1. Original

In [ ]:
best_hp_indo_2_ori = run_optuna_for_model(
    model_name=mod_indo_lite_p1,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    n_trials=20,
    class_weights=class_weights,
    save_root=SAVE_ROOT,
    dataset_name="original"
)

In [ ]:
res_indo_2_ori = pipeline(
    model_name=mod_indo_lite_p1,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    best_hp=best_hp_indo_2_ori,
    class_weights=class_weights,
    dataset_name="original",
    save_root=SAVE_ROOT,
    device=DEVICE,
    num_labels=NUM_LABELS
)

In [ ]:
hp_importance_graph(
    save_root=SAVE_ROOT,
    model_name=mod_indo_lite_p1,
    dataset_name="original"
)

### 4.3.2. ROS

In [ ]:
best_hp_indo_2_ros = run_optuna_for_model(
    model_name=mod_indo_lite_p1,
    X_train=X_train_ros,
    y_train=y_train_ros,
    X_val=X_val,
    y_val=y_val,
    n_trials=20,
    class_weights=class_weights_ros,
    save_root=SAVE_ROOT,
    dataset_name="ROS"
)

In [ ]:
res_indo_2_ros = pipeline(
    model_name=mod_indo_lite_p1,
    X_train=X_train_ros,
    y_train=y_train_ros,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    best_hp=best_hp_indo_2_ros,
    class_weights=class_weights_ros,
    dataset_name="ROS",
    save_root=SAVE_ROOT,
    device=DEVICE,
    num_labels=NUM_LABELS
)

In [ ]:
hp_importance_graph(
    save_root=SAVE_ROOT,
    model_name=mod_indo_lite_p1,
    dataset_name="ROS"
)

### 4.3.3. ROS+NCL

In [ ]:
best_hp_indo_2_ros_ncl = run_optuna_for_model(
    model_name=mod_indo_lite_p1,
    X_train=X_train_ros_ncl,
    y_train=y_train_ros_ncl,
    X_val=X_val,
    y_val=y_val,
    n_trials=20,
    class_weights=class_weights_ros_ncl,
    save_root=SAVE_ROOT,
    dataset_name="ROS-NCL"
)

In [ ]:
res_indo_2_ros_ncl = pipeline(
    model_name=mod_indo_lite_p1,
    X_train=X_train_ros_ncl,
    y_train=y_train_ros_ncl,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    best_hp=best_hp_indo_2_ros_ncl,
    class_weights=class_weights_ros_ncl,
    dataset_name="ROS-NCL",
    save_root=SAVE_ROOT,
    device=DEVICE,
    num_labels=NUM_LABELS
)

In [ ]:
hp_importance_graph(
    save_root=SAVE_ROOT,
    model_name=mod_indo_lite_p1,
    dataset_name="ROS-NCL"
)

## 4.4. indobenchmark/indobert-lite-base-p2

In [ ]:
mod_indo_lite_p2 = "indobenchmark/indobert-lite-base-p2"

### 4.4.1. Original

In [ ]:
best_hp_indo_ori = run_optuna_for_model(
    model_name=mod_indo_lite_p2,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    n_trials=20,
    class_weights=class_weights,
    save_root=SAVE_ROOT,
    dataset_name="original"
)

In [ ]:
res_indo_ori = pipeline(
    model_name=mod_indo_lite_p2,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    best_hp=best_hp_indo_ori,
    class_weights=class_weights,
    dataset_name="original",
    save_root=SAVE_ROOT,
    device=DEVICE,
    num_labels=NUM_LABELS
)

In [ ]:
hp_importance_graph(
    save_root=SAVE_ROOT,
    model_name=mod_indo_lite_p2,
    dataset_name="original"
)

### 4.4.2. ROS

In [ ]:
best_hp_indo_ros = run_optuna_for_model(
    model_name=mod_indo_lite_p2,
    X_train=X_train_ros,
    y_train=y_train_ros,
    X_val=X_val,
    y_val=y_val,
    n_trials=20,
    class_weights=class_weights_ros,
    save_root=SAVE_ROOT,
    dataset_name="ROS"
)

In [ ]:
res_indo_ros = pipeline(
    model_name=mod_indo_lite_p2,
    X_train=X_train_ros,
    y_train=y_train_ros,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    best_hp=best_hp_indo_ros,
    class_weights=class_weights_ros,
    dataset_name="ROS",
    save_root=SAVE_ROOT,
    device=DEVICE,
    num_labels=NUM_LABELS
)

In [ ]:
hp_importance_graph(
    save_root=SAVE_ROOT,
    model_name=mod_indo_lite_p2,
    dataset_name="ROS"
)

In [ ]:
hp_importance_graph(
    save_root=SAVE_ROOT,
    model_name=mod_indo_lite_p2,
    dataset_name="ROS"
)

### 4.4.3. ROS+NCL

In [ ]:
best_hp_indo_ros_ncl = run_optuna_for_model(
    model_name=mod_indo_lite_p2,
    X_train=X_train_ros_ncl,
    y_train=y_train_ros_ncl,
    X_val=X_val,
    y_val=y_val,
    n_trials=20,
    class_weights=class_weights_ros_ncl,
    save_root=SAVE_ROOT,
    dataset_name="ROS-NCL"
)

In [ ]:
res_indo_ros_ncl = pipeline(
    model_name=mod_indo_lite_p2,
    X_train=X_train_ros_ncl,
    y_train=y_train_ros_ncl,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    best_hp=best_hp_indo_ros_ncl,
    class_weights=class_weights_ros_ncl,
    dataset_name="ROS-NCL",
    save_root=SAVE_ROOT,
    device=DEVICE,
    num_labels=NUM_LABELS
)

In [ ]:
hp_importance_graph(
    save_root=SAVE_ROOT,
    model_name=mod_indo_lite_p2,
    dataset_name="ROS-NCL"
)

In [ ]:
hp_importance_graph(
    save_root=SAVE_ROOT,
    model_name=mod_indo_lite_p2,
    dataset_name="ROS-NCL"
)

## 4.5. LazarusNLP/NusaBERT-base

In [ ]:
mod_nusa = "LazarusNLP/NusaBERT-base"

### 4.5.1. Original

In [ ]:
best_hp_nusa_ori = run_optuna_for_model(
    model_name=mod_nusa,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    n_trials=20,
    class_weights=class_weights,
    save_root=SAVE_ROOT,
    dataset_name="original"
)

In [ ]:
res_nusa_ori = pipeline(
    model_name=mod_nusa,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    best_hp=best_hp_nusa_ori,
    class_weights=class_weights,
    dataset_name="original",
    save_root=SAVE_ROOT,
    device=DEVICE,
    num_labels=NUM_LABELS
)

In [ ]:
hp_importance_graph(
    save_root=SAVE_ROOT,
    model_name=mod_nusa,
    dataset_name="original"
)

### 4.5.2. ROS

In [ ]:
best_hp_nusa_ros = run_optuna_for_model(
    model_name=mod_nusa,
    X_train=X_train_ros,
    y_train=y_train_ros,
    X_val=X_val,
    y_val=y_val,
    n_trials=20,
    class_weights=class_weights_ros,
    save_root=SAVE_ROOT,
    dataset_name="ROS"
)

In [ ]:
res_nusa_ros = pipeline(
    model_name=mod_nusa,
    X_train=X_train_ros,
    y_train=y_train_ros,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    best_hp=best_hp_nusa_ros,
    class_weights=class_weights_ros,
    dataset_name="ROS",
    save_root=SAVE_ROOT,
    device=DEVICE,
    num_labels=NUM_LABELS
)

In [ ]:
hp_importance_graph(
    save_root=SAVE_ROOT,
    model_name=mod_nusa,
    dataset_name="ROS"
)

### 4.3.3. ROS+NCL

In [ ]:
best_hp_nusa_ros_ncl = run_optuna_for_model(
    model_name=mod_nusa,
    X_train=X_train_ros_ncl,
    y_train=y_train_ros_ncl,
    X_val=X_val,
    y_val=y_val,
    n_trials=20,
    class_weights=class_weights_ros_ncl,
    save_root=SAVE_ROOT,
    dataset_name="ROS-NCL"
)

In [ ]:
res_nusa_ros_ncl = pipeline(
    model_name=mod_nusa,
    X_train=X_train_ros_ncl,
    y_train=y_train_ros_ncl,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    best_hp=best_hp_nusa_ros_ncl,
    class_weights=class_weights_ros_ncl,
    dataset_name="ROS-NCL",
    save_root=SAVE_ROOT,
    device=DEVICE,
    num_labels=NUM_LABELS
)

In [ ]:
hp_importance_graph(
    save_root=SAVE_ROOT,
    model_name=mod_nusa,
    dataset_name="ROS-NCL"
)